<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа № 5**  
## **Глубокое обучение для табличных данных и временных рядов**

### **1. Цель и задачи работы**

**Цель работы** — сформировать у обучающегося целостное понимание возможностей и ограничений моделей глубокого обучения при работе с **табличными данными**, включая как **стандартные задачи машинного обучения** (регрессия, классификация), так и **специфические сценарии временных рядов**. Работа направлена на освоение методологии проектирования, настройки, диагностики и интерпретации глубоких архитектур в условиях структурированных данных, где доминируют числовые и категориальные признаки, а временная зависимость может присутствовать или отсутствовать.

**Основные задачи работы:**  
1. Использовать два типа данных:  
   - **Табличный датасет из Практической работы № 1** (временной ряд, преобразованный в регрессионную задачу с лагами и инженерными признаками);  
   - **Дополнительный табличный датасет без временной структуры** (например, из репозитория UCI или Kaggle: регрессия — Boston, California Housing; классификация — Adult, Bank Marketing).  
2. Реализовать и сравнить следующие архитектуры глубокого обучения:  
   - **MLP (многослойный перцептрон)** — базовая архитектура для табличных данных;  
   - **1D-CNN** — применённая к последовательности лагов (для временных рядов) или к эмбеддингам категориальных признаков;  
   - **RNN-семейство**: Vanilla RNN, LSTM, GRU — только для задач с временной структурой;  
   - **Двунаправленные модели**: BiLSTM, BiGRU — для offline-анализа временных рядов;  
   - **Encoder–Decoder** — для многопшагового прогнозирования временных рядов;  
   - **Гибридные архитектуры**:  
     - **Tabular + Temporal**: MLP для статических признаков + LSTM для временных;  
     - **CNN + RNN**: извлечение локальных паттернов → моделирование динамики;  
     - **Residual MLP/LSTM**: с skip-соединениями для стабилизации обучения.  
3. Для каждой модели и каждого датасета:  
   - Обеспечить корректную предобработку (масштабирование числовых признаков, эмбеддинги категориальных);  
   - Применить **строгое разделение данных** (для временных рядов — `TimeSeriesSplit`; для табличных — стратифицированное разбиение);  
   - Настроить гиперпараметры через **валидацию без утечки будущего**.  
4. Оценить качество:  
   - Для регрессии: MAE, RMSE, MASE, R²;  
   - Для классификации: Accuracy, F1, AUC-ROC.  
5. Провести **углублённую диагностику обучения**:  
   - Графики train/val loss и метрик;  
   - Анализ переобучения;  
   - Мониторинг градиентов и норм весов.  
6. Исследовать **интерпретируемость**:  
   - Для MLP/CNN — Saliency Maps, Integrated Gradients;  
   - Для RNN/LSTM — анализ скрытых состояний и вентилей;  
   - Для гибридов — вклад временных vs. статических признаков.  
7. Сравнить все глубокие модели между собой и с **лучшими классическими моделями** из Практической работы № 3 (LightGBM, XGBoost, Ridge и др.).  
8. Оценить **вычислительную эффективность**: время обучения, потребление памяти, скорость инференса.  
9. Разработать **модульную кодовую базу** на PyTorch или TensorFlow с поддержкой переключения между задачами (табличная/временная) и архитектурами.  
10. Опубликовать воспроизводимый пайплайн: данные, конфигурации, веса, визуализации, сравнительные таблицы.  

---

### **2. Теоретические предпосылки**

На протяжении десятилетий табличные данные оставались доменом классических алгоритмов — деревьев решений, ансамблей и линейных моделей. Глубокое обучение долгое время считалось неэффективным в этой области из-за:  
- отсутствия пространственной или последовательной структуры, на которой строятся CNN/RNN;  
- высокого риска переобучения при малом объёме данных;  
- сложности работы с разнородными типами признаков (числовые, категориальные, разреженные).

Однако последние годы показали, что при правильной архитектуре и регуляризации **глубокие модели могут конкурировать, а иногда и превосходить градиентный бустинг**, особенно в задачах с:  
- длинными последовательностями (временные ряды);  
- сложными нелинейными взаимодействиями признаков;  
- наличием мультимодальных или иерархических структур.

**MLP**, несмотря на простоту, остаётся сильной базовой линией для табличных данных. С добавлением dropout, batch normalization и внимательной инициализации он демонстрирует удивительную устойчивость.

**1D-CNN** находят применение двумя способами:  
- как **свёртки по временной оси** — для извлечения локальных паттернов в лагах;  
- как **свёртки по эмбеддингам категориальных признаков**, выстроенных в искусственную «последовательность».

**Рекуррентные сети (LSTM, GRU)** применимы исключительно в контексте **временной или упорядоченной структуры данных**. Их способность моделировать долгосрочные зависимости делает их незаменимыми в задачах прогнозирования, но бесполезными в чисто табличных задачах без порядка.

**Encoder–Decoder** архитектура — естественное решение для **многопшагового прогнозирования**, где требуется сгенерировать последовательность выходов. В отличие от рекурсивной стратегии, она обучается end-to-end и избегает накопления ошибки.

**Гибридные модели** открывают путь к объединению статического и динамического контекста: например, прогнозирование спроса с учётом как исторических продаж (LSTM), так и характеристик товара и магазина (MLP).

Ключевой тезис настоящей работы: **глубокое обучение для табличных данных — это не универсальный инструмент, а контекстно-зависимый выбор, требующий понимания природы данных и задачи**.

---

### **3. Порядок выполнения работы**

#### **3.1. Подготовка данных**

- **Датасет 1 (временной ряд)**: финальная таблица из Практической работы № 1 (целевая переменная + лаги + инженерные признаки);  
- **Датасет 2 (табличный)**: например, `California Housing` (регрессия) или `Adult Income` (классификация);  
- Предобработка:  
  - Числовые признаки → масштабирование (StandardScaler/RobustScaler);  
  - Категориальные → эмбеддинги или one-hot (в зависимости от архитектуры);  
  - Для временных рядов — формирование окон без утечки будущего.

#### **3.2. Реализация архитектур**

- Все модели реализуются в едином интерфейсе (`BaseTabularModel`);  
- Поддержка GPU и CPU;  
- Для временных рядов — вход в формате `(batch, seq_len, features)`;  
- Для табличных — `(batch, num_features)`.  

Примеры:  
- **MLP**: `Linear → ReLU → Dropout → ... → Output`;  
- **1D-CNN**: `Conv1d → ReLU → AdaptiveAvgPool1d → Flatten → Linear`;  
- **LSTM**: `LSTM → Dropout → Linear`;  
- **Encoder–Decoder**: отдельные модули для кодирования и декодирования последовательности.

#### **3.3. Обучение и валидация**

- **Временные ряды**: разделение `train = до T−180`, `val = T−180:T−90`, `test = T−90:T`; валидация через временные окна;  
- **Табличные данные**: 80/10/10, стратификация по целевой переменной;  
- **Оптимизация**: Adam, начальный learning rate = 1e−3, ReduceLROnPlateau;  
- **Регуляризация**: dropout (0.2–0.5), weight decay, gradient clipping (для RNN);  
- **Остановка**: early stopping по val loss (patience = 10).

#### **3.4. Эксперименты и оценка**

- Для каждой модели и датасета:  
  - Запуск с фиксированным seed;  
  - Сохранение лучшей эпохи по валидации;  
  - Оценка на тесте — **только один раз**;  
- Метрики: MAE, RMSE, R² (регрессия); Accuracy, F1 (классификация);  
- Время обучения (сек), память (MB), время инференса (мс на наблюдение).

#### **3.5. Интерпретация и диагностика**

- **Saliency Maps**: градиенты прогноза по входным признакам;  
- **Integrated Gradients**: более устойчивая оценка вклада признаков;  
- **Для LSTM/GRU**: визуализация forget/input gates на реальных последовательностях;  
- **Скрытые состояния**: кластеризация с помощью t-SNE — отражают ли они семантику данных?

#### **3.6. Сравнение и выводы**

- Сравнительная таблица: «Модель × Датасет × Метрика × Время × Интерпретируемость»;  
- Ранжирование по взвешенной оценке;  
- Ответ на вопрос: **когда глубокое обучение оправдано для табличных данных?**  
- Сравнение с бустингом (LightGBM/XGBoost) — лидерами в классическом ML.

#### **3.7. Публикация и воспроизводимость**

- Код — в публичном GitHub/GitLab репозитории;  
- Данные — под лицензией CC BY 4.0;  
- Веса моделей, логи обучения, конфигурации — архивированы;  
- Dockerfile или `requirements.txt` для воспроизведения;  
- При необходимости — развёртывание демо на Hugging Face Spaces.

---

### **4. Дополнительные исследовательские задания**

1. **Сравнение MLP с табличными специализированными архитектурами**: NODE, TabNet, DCN (без реализации, через библиотеки);  
2. **Анализ влияния размера датасета**: при каком количестве наблюдений DL начинает выигрывать у бустинга?  
3. **Робастность к зашумлённым признакам**: сравнение устойчивости MLP, LSTM и LightGBM к добавлению шума;  
4. **Transfer learning между табличными задачами**: возможно ли?  
5. **Ансамблирование DL и классических моделей**: stacking MLP + LightGBM.

---

### **5. Требования к отчёту**

Отчёт оформляется в соответствии с ГОСТ 7.32–2017 и включает:  
1. **Введение** — эволюция DL для табличных данных, мотивация, связь с предыдущими работами;  
2. **Методология** — описание датасетов, архитектур, стратегий обучения и оценки;  
3. **Результаты** — таблицы, графики обучения, интерпретационные визуализации, сравнение с ML;  
4. **Обсуждение** — когда и почему использовать MLP, LSTM или CNN для табличных данных; анализ компромиссов;  
5. **Заключение** — практические рекомендации по выбору архитектуры в зависимости от типа данных, объёма и задачи;  
6. **Список литературы** — с ссылками на оригинальные работы по LSTM, GRU, TabNet, и современные обзоры по DL для табличных данных;  
7. **Приложения** — фрагменты кода, архитектурные схемы, скриншоты визуализаций, сравнительные метрики.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Реализованы все архитектуры на двух типах данных; корректная предобработка и разделение; глубокая диагностика и интерпретация; сравнение с классическим ML; модульный код; полная воспроизводимость; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 6–8 моделей на обоих датасетах; есть метрики и базовая интерпретация; отсутствуют лишь отдельные элементы (например, гибридные архитектуры или сравнение с ML). |
| **Удовлетворительно** | Работа проведена только на одном типе данных (только временные ряды или только табличные); реализованы 3–5 моделей; есть прогноз и метрики, но без диагностики. |
| **Неудовлетворительно** | Отсутствует сравнительный анализ, данные не соответствуют требованиям, нет разделения на выборки, отчёт не представлен. |
